## Shape Detection

here we can see it will detect one shape properly in any image

In [ ]:
import cv2
import numpy as np
import math

def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print("Image not found or path is incorrect!")
        exit()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 1)
    edges = cv2.Canny(blurred, 50, 150)
    cv2.imshow("Edges", edges)
    return img, edges

def find_contours(edges):
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0 or hierarchy is None:
        print("No contours or hierarchy found!")
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        exit()
    
    # Sort contours by area and select the two largest ones
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]  # Taking top 5 for flexibility
    print(f"Total Contours Found: {len(contours)}")
    return contours

def detect_shape(cnt):
    epsilon = 0.02 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    vertices = len(approx)

    M = cv2.moments(cnt)
    if M['m00'] != 0:
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])
    else:
        cx, cy = 0, 0
    center = (cx, cy)

    area = cv2.contourArea(cnt)

    side_lengths = [
        math.hypot(approx[i][0][0] - approx[(i + 1) % vertices][0][0],
                   approx[i][0][1] - approx[(i + 1) % vertices][0][1])
        for i in range(vertices)
    ]

    angles = []
    for i in range(vertices):
        p0, p1, p2 = approx[i - 1][0], approx[i][0], approx[(i + 1) % vertices][0]
        v1 = (p0[0] - p1[0], p0[1] - p1[1])
        v2 = (p2[0] - p1[0], p2[1] - p1[1])
        dot_prod = v1[0] * v2[0] + v1[1] * v2[1]
        mag1 = math.hypot(*v1)
        mag2 = math.hypot(*v2)
        angle = 0 if mag1 == 0 or mag2 == 0 else math.acos(max(min(dot_prod / (mag1 * mag2), 1), -1)) * 180 / math.pi
        angles.append(angle)

    shape = classify_shape(vertices, side_lengths, angles, cnt, area)
    return {'shape': shape, 'center': center, 'area': area, 'contour': approx, 'sides': side_lengths, 'angles': angles}

def classify_shape(vertices, side_lengths, angles, cnt, area):
    if vertices == 3:
        return "Triangle"
    elif vertices == 4:
        sides_equal = all(abs(side - side_lengths[0]) < 10 for side in side_lengths)
        angles_close_90 = all(abs(angle - 90) < 10 for angle in angles)
        if sides_equal and angles_close_90:
            return "Square"
        elif angles_close_90:
            return "Rectangle"
        else:
            return "Quadrilateral"
    elif vertices > 4:
        circularity = 4 * math.pi * area / (cv2.arcLength(cnt, True) ** 2)
        return "Circle" if circularity > 0.8 else f"{vertices}-gon"
    return "Unknown"

def draw_shapes(img, shape_data_list):
    for shape_data in shape_data_list:
        cv2.drawContours(img, [shape_data['contour']], -1, (0, 255, 0), 2)
        cv2.putText(img, shape_data['shape'], shape_data['center'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        print(f"Detected Shape: {shape_data['shape']} at {shape_data['center']}")
    cv2.imshow('Detected Shapes', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def main():
    image_path = r'IMGTEST/003.png'
    img, edges = load_and_preprocess_image(image_path)
    contours = find_contours(edges)

    # Detect shapes from the two largest contours
    shape_data_list = [detect_shape(cnt) for cnt in contours[:2]]
    if len(shape_data_list) == 0:
        print("No valid shapes detected.")
        return

    draw_shapes(img, shape_data_list)

if __name__ == "__main__":
    main()
